In [1]:
import pandas as pd
data =  pd.read_csv('krkopt.data', sep=",",engine='python', header=None)
print(data)

       0  1  2  3  4  5        6
0      a  1  b  3  c  2     draw
1      a  1  c  1  c  2     draw
2      a  1  c  1  d  1     draw
3      a  1  c  1  d  2     draw
4      a  1  c  2  c  1     draw
...   .. .. .. .. .. ..      ...
28051  b  1  g  7  e  5  sixteen
28052  b  1  g  7  e  6  sixteen
28053  b  1  g  7  e  7  sixteen
28054  b  1  g  7  f  5  sixteen
28055  b  1  g  7  g  5  sixteen

[28056 rows x 7 columns]


In [2]:
# Lettre = {'a':10,'b':20,'c':30,'d':40,'e':50,'f':60,'g':70,'h':80}
Lettre = {'a':0.1,'b':0.2,'c':0.3,'d':0.4,'e':0.5,'f':0.6,'g':0.7,'h':0.8} #Meilleur résultat
WK=[]
WR=[]
BK=[]

for i in range(len(data.iloc[:,0])):
    WK.append(Lettre[data.iloc[i,0]] + data.iloc[i,1])
    WR.append(Lettre[data.iloc[i,2]] + data.iloc[i,3])
    BK.append(Lettre[data.iloc[i,4]] + data.iloc[i,5])
    
data2 = pd.DataFrame({'WhiteKing':WK,'WhiteRook':WR,'BlackKing':BK,'Results': data.iloc[:,6]})
data2
    

,WhiteKing,WhiteRook,BlackKing,Results
0,1.1,3.2,2.3,draw
1,1.1,1.3,2.3,draw
2,1.1,1.3,1.4,draw
3,1.1,1.3,2.4,draw
4,1.1,2.3,1.3,draw
...,...,...,...,...
28051,1.2,7.7,5.5,sixteen
28052,1.2,7.7,6.5,sixteen
28053,1.2,7.7,7.5,sixteen
28054,1.2,7.7,5.6,sixteen


In [3]:
from sklearn import datasets
import numpy as np

X = data2.iloc[:, [0,1,2]]
y = data2.iloc[:,3]
X = X.to_numpy() #turn to ndarray
y = y.to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

### Arbre de décision

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree = DecisionTreeClassifier(criterion='gini', splitter='best')
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.68
Erreur quadratique: 1.18


In [6]:
Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
Result

,Prediction,Y_test
0,thirteen,fourteen
1,thirteen,thirteen
2,fourteen,fourteen
3,five,five
4,eleven,nine
...,...,...
8412,thirteen,thirteen
8413,eleven,eleven
8414,nine,nine
8415,fifteen,fifteen


In [21]:
def erreur_quadratique(Result):
    dico = {'draw':0,'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9,'ten':10,'eleven':11,'twelve':12,'thirteen':13,'fourteen':14,'fifteen':15,'sixteen':16}
    Pred = []
    Test = []
    n = Result.shape[0]
    s = 0
    for i in range(Result.shape[0]) :
        clef0 = Result.iloc[i,0]
        clef1 = Result.iloc[i,1]
        if clef0 == 'draw' and clef1 == 'draw':
            Pred.append(dico[clef0])
            Test.append(dico[clef1])
        elif clef0 == 'draw' or clef1 == 'draw':
            n = n - 1
        else : 
            Pred.append(dico[clef0])
            Test.append(dico[clef1])
        
    for i in range(len(Pred)):
        s = s + (Pred[i] - Test[i])
    
    return(s**2/n)

In [13]:
dico = {'draw':'error','zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9,'ten':10,'eleven':11,'twelve':12,'thirteen':13,'fourteen':14,'fifteen':15,'sixteen':16}

In [22]:
erreur_quadratique(Result)

1.2559658377292138

### Arbre de désision v2

In [35]:
tree = DecisionTreeClassifier(criterion='entropy')

tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.67
Erreur quadratique: 0.22


### Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

rdforst = RandomForestClassifier()

rdforst = rdforst.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.67
Erreur quadratique: 0.22


### K Plus Proches Voisins

In [36]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.49
Erreur quadratique: 803.63


### Perceptron

In [38]:
from sklearn.linear_model import Perceptron

ppn = Perceptron()
ppn.fit(X_train, y_train)
y_pred = ppn.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.21
Erreur quadratique: 12356.33


### Régression logistique

In [39]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

C:\Users\Andy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.25
Erreur quadratique: 5816.52


### Support vector machines

In [40]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))


Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.33
Erreur quadratique: 2762.05


### Centrer Réduit + Arbre de Décision

In [30]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [32]:
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(criterion='gini', splitter='best', )
tree.fit(X_train_std, y_train)
y_pred = tree.predict(X_test_std)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.68
Erreur quadratique: 0.91


In [33]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=17, p=2, metric='minkowski')
knn.fit(X_train_std, y_train)
y_pred = knn.predict(X_test_std)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Result = pd.DataFrame({'Prediction':y_pred,'Y_test':y_test})
print('Erreur quadratique: %.2f' % erreur_quadratique(Result))

Accuracy: 0.49
Erreur quadratique: 891.29


On obtient naturellement la même précision avec un arbre car on ne change pas l'ordre en centrant réduisant.

### Brouillon

In [34]:
a=0
b=0
c=0

for i in range(Result.shape[0]):
    clef0 = Result.iloc[i,0]
    clef1 = Result.iloc[i,1]

    if clef0 == 'draw' and clef1 == 'draw':
        a = a +1
    elif clef0 == 'draw' or clef1 == 'draw':
        b = b + 1
    else : 
        c = c +1
        
print(a,b,c, a+b+c,Result.shape[0])


749 733 6935 8417 8417
